In [162]:
import sys
import pandas as pd
import numpy as np

data = pd.read_csv('./train.csv', encoding = 'big5')
# print(data)
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
# print(data)
# print(raw_data)

In [163]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample
# print(month_data)

In [164]:
x = np.empty([12 * 471, 18 * 9], dtype = float) #traning
y = np.empty([12 * 471, 1], dtype = float) #testing
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
# print(x)
# print(y)
# df = pd.DataFrame(x)
# df.to_csv('file.csv',index=False)

In [165]:
#scaling
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
# print(x)
# print(len(x))


In [166]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
# print(x_train_set)
# print(y_train_set)
# print(x_validation)
# print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

4521
4521
1131
1131


In [167]:
dim = 18 * 9 + 1 #18 features and 9 hours, 1 bias
w = np.ones([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 10000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
# w #(18*9,1)

0:51.65880600054525
100:20.77437564222041
200:14.127460058537887
300:11.303857670788975
400:9.75735463757443
500:8.790692587072344
600:8.132230172649818
700:7.657473854962997
800:7.301552728315731
900:7.027160886807602
1000:6.811177734873491
1100:6.638413442067585
1200:6.498417831545401
1300:6.383737204109007
1400:6.288904904313378
1500:6.209824896717985
1600:6.143376719550692
1700:6.087151427316504
1800:6.039268829370359
1900:5.998247451793656
2000:5.962910016891086
2100:5.932313603545214
2200:5.905697380038531
2300:5.882443083695772
2400:5.862044880037761
2500:5.844086201253044
2600:5.828221825181654
2700:5.8141639193771475
2800:5.801671105507553
2900:5.790539838697473
3000:5.780597571519529
3100:5.771697301555401
3200:5.76371319747139
3300:5.7565370703528185
3400:5.750075511024628
3500:5.744247554882434
3600:5.738982766741748
3700:5.734219661855867
3800:5.729904397378539
3900:5.72598968250727
4000:5.722433866343131
4100:5.719200170896092
4200:5.716256043216149
4300:5.713572605766973

In [168]:
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)): #scaling
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)

# print(test_x)
# df = pd.DataFrame(test_x)
# df.to_csv('1file',index=False)


<ipython-input-168-22ca531fe321>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[test_data == 'NR'] = 0


In [169]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
# ans_y

In [170]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 6.675990980112423]
['id_1', 18.071674932253497]
['id_2', 24.038225880811613]
['id_3', 7.772257990398251]
['id_4', 27.02666876933244]
['id_5', 22.28559302807508]
['id_6', 23.89889844954017]
['id_7', 30.41920510163969]
['id_8', 16.86218519402351]
['id_9', 59.60706265479261]
['id_10', 11.721145798998077]
['id_11', 9.26896616337624]
['id_12', 63.277172907766975]
['id_13', 53.28392517194697]
['id_14', 22.196853423304336]
['id_15', 12.170905086948878]
['id_16', 32.54140828422534]
['id_17', 66.68114491778044]
['id_18', -0.4288923754744491]
['id_19', 17.08174420079968]
['id_20', 41.78232474057129]
['id_21', 72.12298475750184]
['id_22', 9.288776353342515]
['id_23', 17.79839755885693]
['id_24', 14.717265726180063]
['id_25', 37.71078647347016]
['id_26', 14.568564764635115]
['id_27', 69.34780229867515]
['id_28', 7.2696073766350695]
['id_29', 55.42394068387627]
['id_30', 24.464710738827392]
['id_31', 8.490068429392881]
['id_32', 2.485893898490215]
['id_33', 18.6196939000320